# PSyclone tutorial: NEMO API Example 3 - OpenMP

This example shows how we can add OpenMP directives to the code using a transformation. The resulting code can be run in parallel on multi-core processors.

Let's continue with the code introduced in example 2 and create a schedule from it:

In [ ]:
code = '''program test
  use timing_mod
  implicit none
  integer, parameter :: jpi=10, jpj=10, jpk=10
  real, allocatable, dimension(:,:,:) :: a,b
  integer :: ji,jj,jk
  allocate(a(jpi,jpj,jpk), b(jpi,jpj,jpk))
  call timer_start()
  do jk=1,jpk
    do jj=1,jpj
      do ji=1,jpi
        b(ji,jj,jk) = 0.0
      end do
    end do
  end do
  do jk=1,jpk
    do jj=1,jpj
      do ji=1,jpi
        a(ji,jj,jk) = b(ji,jj,jk)
      end do
    end do
  end do
  call timer_end()
  write (6,*) "HELLO"
end program test'''

In [ ]:
from fparser.common.readfortran import FortranStringReader
from fparser.two.symbol_table import SYMBOL_TABLES
reader = FortranStringReader(code)
from fparser.two.parser import ParserFactory
parser = ParserFactory().create(std="f2003")
parse_tree = parser(reader)

from psyclone.psyGen import PSyFactory
psy = PSyFactory("nemo").create(parse_tree)

invoke = psy.invokes.invoke_list[0]
schedule = invoke.schedule

print(schedule.view())

Now that we have created the PSyIR representation of the code we apply a PSyclone OpenMP transformation. The transformation adds OpenMP nodes around all loops that PSyclone has determined to be over latitude. Notice that the transformation also checks whether the loop contains a kernel. If not this loop is not parallelised. This avoids parallelising loops without any relevant computation in them (not strictly required in this particular example).

In [ ]:
from psyclone.transformations import OMPParallelLoopTrans
from psyclone.nemo import NemoKern
omp_trans = OMPParallelLoopTrans()
for loop in schedule.loops():
    kernels = loop.walk(NemoKern)
    if kernels and loop.loop_type == "lat":
        omp_trans.apply(loop)

Taking a look at what has happened to the PSyIR representation you can see that new OpenMP nodes have been added in the appropriate places.

In [ ]:
print(schedule.view())

If we've finished with our transformations we can write out the resultant code which can now run in parallel using OpenMP. Notice that there are some clauses in the OpenMP directives including declarations of private variables. PSyclone works out what these should be for correct execution so you don't need to worry about them.

The [transformation](https://psyclone.readthedocs.io/en/stable/transformations.html#available-transformations) provides an optional argument allowing you to set a schedule other than `static`.

In [ ]:
print(psy.gen)

You might like to try to change the application of the transformation so that the parallelisation is over levels. You could also try to change the schedule. To do this you will need to look at the PSyclone [user guide](https://psyclone.readthedocs.io/en/stable/transformations.html#available-transformations) to find the necessary optional argument to the transformation. (Note, it's best to start with an unmodified schedule so this has been done for you below. The symbol tables associated with the Fortran parser must also be cleared.)

In [ ]:
reader = FortranStringReader(code)
SYMBOL_TABLES.clear()
parse_tree = parser(reader)
psy = PSyFactory("nemo").create(parse_tree)
invoke = psy.invokes.invoke_list[0]
schedule = invoke.schedule

for loop in schedule.loops():
    print("Your code should replace this print")

print(schedule.view())

Let's move on to the [next example](nemo_example4.ipynb)...